<a href="https://colab.research.google.com/github/dolmani38/Summary/blob/master/korean_abstractive_summarizaion_v1.2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Korean Summarizer Using Multiple Discriminators**

참조 : https://www.kaggle.com/shivamb/beginners-guide-to-text-generation-using-lstms

참조 : https://github.com/williamSYSU/TextGAN-PyTorch

* 2020년12월27일 v1.0 완전히 실패...
* Generator 새로 제작!

In [1]:
DO_ALL = True # 전체 실행하면서 시간 걸리는 걸 Pass 하려면 이걸 False ...


In [2]:

if DO_ALL:
    !pip install sentence-transformers==0.3.0
    !pip install transformers==3.0.2
    !pip install wikipedia
    !pip install konlpy

     |████████████████████████████████| 71kB 3.4MB/s 
     |████████████████████████████████| 1.5MB 8.1MB/s 
     |████████████████████████████████| 890kB 37.0MB/s 
     |████████████████████████████████| 2.9MB 29.2MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.0-cp36-none-any.whl size=86754 sha256=ed0f7fa9fc54d6e7a87ab2e20470d19e85f28764d5d948f0f9090619332d4699
  Stored in directory: /root/.cache/pip/wheels/b8/23/85/85d6a9a6c68f0625a1ecdaad903bb0a78df058c10cf74f9de4
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=5161918000e17ba1ddc626312490ffce2e720f02bcff10438c08929cafb54e95
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses
     |████████████████████████████████| 778kB 5.8MB/s 
     |████████████████████████████████| 3.0MB 37.1MB/s 
     |████████████████████████████████| 1.1MB 39.8MB/s 
  Fo

In [3]:
# keras module for building LSTM 
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Embedding, LSTM, Dense, Dropout
from keras.preprocessing.text import Tokenizer
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from tensorflow.python.framework import tensor_shape
import keras.utils as ku 

# set seeds for reproducability
from tensorflow.random import set_seed
from numpy.random import seed
set_seed(2)
seed(1)

import pandas as pd
import numpy as np
import string, os 

import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)

# 학습을 위한 데이터 준비

네이버 뉴스에서 아무거나 하나 Text를 얻어옴

이것을 '요약' 목표

In [4]:
org_text = """
옛날 어느 집에 귀여운 여자 아기가 태어났어요.

아기는 무럭무럭 자라서, 예쁘고 마음씨 고운 소녀가 되었어요.

그러던 어느날, 소녀의 어머니가 병이들어

그만 세상을 떠나고 말았어요.

 

소녀의 아버지는 홀로 남은 소녀가 걱정되었어요.

그래서 얼마 후 새어머니를 맞이했어요.

새어머니는 소녀보다 나이가 위인 두 딸을 데리고 왔어요.

그러나 새어머니와 언니들은 성질이 고약한 심술쟁이들이었어요.

새어머니는 소녀가 자기 딸들보다 예쁘고 착한 게 못마땅했어요.

 

그런데 이번에는 아버지마저 돌아가셨어요.

소녀는 하녀처럼 하루 종일 쓸고, 닦고, 집안일을 도맡아 했어요.

해도 해도 끝이 없는 집안일이 힘들어 지칠때면

난롯가에 앉아서 잠시 쉬곤 했지요.

 

"엄마, 저애를 신데렐라라고 불러야겠어요."

"온통 재투성이잖아요. 호호호!" 두 언니는 소녀를 놀려 댔어요.

 

어느 날, 왕궁에서 무도회가 열렸어요.

신데렐라의 집에도 초대장이 왔어요.

 

새어머니는 언니들을 데리고 무도회장으로 떠났어요.

신데렐라도 무도회에 가고 싶었어요.

혼자 남은 신데렐라는 훌쩍훌쩍 울기 시작했어요.

"신데렐라, 너도 무도회에 가고 싶니?"

신데렐라가 고개를 들어보니, 마법사 할머니가 빙그레 웃고 있었어요.

 

"내가 너를 무도회에 보내주마

호박 한개와 생쥐 두마리, 도마뱀을 구해 오렴."

  

마법사 할머니가 주문을 외웠어요.

그리고 지팡이로 호박을 건드리자, 호박이 화려한 황금 마차로 변했어요.

이번에는 생쥐와 도마뱀을 건드렸어요.

그랬더니 생쥐는 흰말로, 도마뱀은 멋진 마부로 변했답니다.

 

신데렐라의 옷도 구슬 장식이 반짝이는 예쁜 드레스로 바뀌웠어요.

"신데렐라, 발을 내밀어 보거라."

할머니는 신데렐라에게 반짝반짝 빛나는 유리 구두를 신겨 주었어요

"신데렐라, 밤 열두시가 되면 모든게 처음대로 돌아간단다.

황금 마차는 호박으로, 흰말은 생쥐로, 마부는 도마뱀으로 변하게 돼.

그러니까 바늗시 밤 열두 시가 되기 전에 돌아와야 해. 알겠지?"

 왕자님도 아름다운 신데렐라에게 마음을 빼았겼어요.

완자님은 무도회장에 모인 다른 아가씨들은 쳐다보지도 않고,

신데렐라하고만 춤을 추었어요.

신데렐라는 왕자님과 춤을 추느라 시간 가는 줄도 몰랐어요.

땡, 땡, 땡...... 벽시계가 열두 시를 알리는 소리에 신데렐라는 화들짝 놀랐어요.

 

신데렐라가 허둥지둥 왕궁을 빠져나가는데,

유리 구두 한 짝이 벗겨졌어요.

하지만 구두를 주울 틈이 없었어요.


신데렐라를 뛰쫓아오던 왕자님은 층계에서 유리 구두 한 짝을 주웠어요.

왕자님은 유리 구두를 가지고 임금님께 가서 말했어요.

"이 유리 구두의 주인과 결혼하겠어요."

그래서 신하들은 유리 구두의 주인을 찾아 온 나라를 돌아다녔어요.

 언니들은 발을 오므려도 보고, 구두를 늘려도 보았지만

한눈에 보기에도 유리 구두는 너무 작았어요.

 

그때, 신데렐라가 조용히 다가와 말했어요.

"저도 한번 신어 볼 수 있나요?"

​신데렐라는 신하게 건넨 유리 구두를 신었어요,

유리 구두는 신데렐라의 발에 꼭 맞았어요.

 

신하들은 신데렐라를 왕궁으로 데리고 갔어요.

 

그 뒤 신데렐라는 완자님과 결혼하여 오래오래 행복하게 살았대요.



"""

In [5]:
org_text

'\n옛날 어느 집에 귀여운 여자 아기가 태어났어요.\n\n아기는 무럭무럭 자라서, 예쁘고 마음씨 고운 소녀가 되었어요.\n\n그러던 어느날, 소녀의 어머니가 병이들어\n\n그만 세상을 떠나고 말았어요.\n\n \n\n소녀의 아버지는 홀로 남은 소녀가 걱정되었어요.\n\n그래서 얼마 후 새어머니를 맞이했어요.\n\n새어머니는 소녀보다 나이가 위인 두 딸을 데리고 왔어요.\n\n그러나 새어머니와 언니들은 성질이 고약한 심술쟁이들이었어요.\n\n새어머니는 소녀가 자기 딸들보다 예쁘고 착한 게 못마땅했어요.\n\n \n\n그런데 이번에는 아버지마저 돌아가셨어요.\n\n소녀는 하녀처럼 하루 종일 쓸고, 닦고, 집안일을 도맡아 했어요.\n\n해도 해도 끝이 없는 집안일이 힘들어 지칠때면\n\n난롯가에 앉아서 잠시 쉬곤 했지요.\n\n \n\n"엄마, 저애를 신데렐라라고 불러야겠어요."\n\n"온통 재투성이잖아요. 호호호!" 두 언니는 소녀를 놀려 댔어요.\n\n \n\n어느 날, 왕궁에서 무도회가 열렸어요.\n\n신데렐라의 집에도 초대장이 왔어요.\n\n \n\n새어머니는 언니들을 데리고 무도회장으로 떠났어요.\n\n신데렐라도 무도회에 가고 싶었어요.\n\n혼자 남은 신데렐라는 훌쩍훌쩍 울기 시작했어요.\n\n"신데렐라, 너도 무도회에 가고 싶니?"\n\n신데렐라가 고개를 들어보니, 마법사 할머니가 빙그레 웃고 있었어요.\n\n \n\n"내가 너를 무도회에 보내주마\n\n호박 한개와 생쥐 두마리, 도마뱀을 구해 오렴."\n\n  \n\n마법사 할머니가 주문을 외웠어요.\n\n그리고 지팡이로 호박을 건드리자, 호박이 화려한 황금 마차로 변했어요.\n\n이번에는 생쥐와 도마뱀을 건드렸어요.\n\n그랬더니 생쥐는 흰말로, 도마뱀은 멋진 마부로 변했답니다.\n\n \n\n신데렐라의 옷도 구슬 장식이 반짝이는 예쁜 드레스로 바뀌웠어요.\n\n"신데렐라, 발을 내밀어 보거라."\n\n할머니는 신데렐라에게 반짝반짝 빛나는 유리 구두를 신겨 주었어요\n\n"신데렐라, 밤 열두시가 되

한국어 문법체계에 따라 요약문을 생성하기 위해 한국어 문장 샘플을 준비

'한글 위키백과'에서 임의의 문장을 수집 함

In [6]:
#한국어 위키백과에서 스크랩핑

import wikipedia as wiki
wiki.set_lang('ko')

In [7]:
# '전래동화' 라는 keyword로 100개 page의 Text를 취득

def __search_from_wiki(question,max_rank):
    results = wiki.search(question,results=max_rank)
    print(results)
    contents = []
    for result in results:
        try:
            page = wiki.page(result)
            #print(f"Top wiki result: {page}")
            text = page.content
            ln = len(text)
            print(f'Collecting page : {page} , text length {str(ln)}')
            #if ln < 4000:
            #  contents.append(text)
            #else:
            #  contents.append(text[0:4000])
            contents.append(text)
        except Exception as ex:
          print(ex)
    return contents

if DO_ALL:
    ko_grammar_set_raw = __search_from_wiki("전래동화", 100)

print(f'전체 수집한 Page Count : {len(ko_grammar_set_raw)}')

['동화', '신 전래동화', '꾸러기 수비대', '아동 문학', '호시조라 미유키', '한국의 사찰', '해와 달이 된 오누이', '잠자는 숲속의 미녀', '거북', '이한갈', '정은찬', '김기두 (배우)', '옛날 옛적에 (애니메이션)', '계룡선녀전 (드라마)', '동요', '이상훈 (1976년)', '최홍일', '육진수 (격투기 선수)', '제비', '장석현 (연예인)', '유다미', '밀교 (불교)', '최지웅', '박재훈 (배우)', '한다은', '재희', '정미남', '안젤리나 다닐로바', '도깨비', '기탄교육', '국지용', '의왕백운호수축제', '박민경', '정정아', '콩딱쿵! 이야기 주머니', '윤기원 (배우)', '선녀와 나무꾼', '남생이', '대장화홍련전', '콩쥐팥쥐 (동음이의)', '토끼전', '미녀와 야수', '지대한', '이서휴게소', '은비까비의 옛날옛적에', '빨간 자전거', '콩쥐팥쥐전', '이설구', '홍석연', '아시리아인', '백조의 호수', '김덕현 (배우)', '티베트', '서예', '도교', '켈트 다신교', '금도끼 은도끼', '장화, 홍련 (동음이의)', '허구 국가', '노상현', '제네시스 (밴드)', '타이완의 문화', '도깨비 (동음이의)', '골디락스', '선녀강림', '손춘익', '자와어', '안동국제탈춤페스티벌', '윌리엄 버틀러 예이츠', '성덕대왕신종', '모모타로', '고려', '한국 문학', '라푼젤 (영화)', '토비트', '조선 후기의 문학', '송월동 동화마을', '홍석천', '계룡선녀전', '이집트', '응우옌 왕조', '안지환', '아시아의 역사', '외래어', '메이플 월드', '프랑스인', '스키타이족', '인도네시아', '이탈리아', 'MBC 창작동요제', '네버랜드', '일본 제국', '프랑크인', '바이킹', '김환영 (작가)', '드랑 나흐 오스텐', '조선', '오윤 (화가)', '원나라', '앱북']
"콩쥐팥쥐 (동음이의)" may refer to: 
콩쥐팥쥐


In [8]:

if DO_ALL:
    ko_grammar_set_raw += __search_from_wiki("역사", 100)

print(f'전체 수집한 Page Count : {len(ko_grammar_set_raw)}')

['역사', '한국의 역사', '불교의 역사', '대체 역사', '일본의 역사', '스위스 역사 사전', '유럽의 역사', '고려의 역사', '세계의 역사', '조선의 역사', '우즈베키스탄의 역사', '동북아역사재단', '중국의 역사', '대한민국의 역사 드라마 목록', '한국 불교의 역사', '서울역사박물관', '오키나와현의 역사', '역사주의', '역사학', '러시아의 역사', '역사적 예수', '다큐멘터리 역사를 찾아서', '소련의 역사', '조선민주주의인민공화국의 역사', '조지아의 역사', '영국의 역사', '역사철학', '프랑스의 역사', '역사신학', '애플의 역사', '인터넷의 역사', '역사저널 그날', '유엔의 역사', '역사지진', '힌두교의 역사', '역사 시대', '독일의 역사', '역사소설', '압하지야의 역사', '아일랜드의 역사', '구글의 역사', '영화의 역사', '기독교의 역사', '운영 체제의 역사', '몬테네그로의 역사', '태국의 역사', '사기 (역사서)', '파키스탄의 역사', '음소문자의 역사', '그리스의 역사', '의사 역사학', '사학자', '베트남의 역사', '기술의 역사', '역사스페셜', '안드로이드 버전 역사', '서울역', '인도의 역사', '미국 육군 역사관', '벨라루스의 역사', '역사언어학', '퀴디치의 역사', '사극', '역사적 유물론', '마리아론의 역사', '역사 (헤로도토스)', 'FC 서울의 역사', '폭력의 역사', '교육의 역사', '중화민국의 역사', '터키의 역사', '몽골의 역사', '근대 그리스의 역사', '스페인의 역사', '베트남 역사박물관', '핀란드의 역사', '우크라이나의 역사', '맨체스터 유나이티드 FC의 역사', '이란의 역사', '음악의 역사', '에스페란토의 역사', '타이완의 역사', '역사 재현', '민자역사', '크림반도의 역사', '라오스의 역사', '제2차 세계 대전 기간 미국의 군사 역사', '브리튼 제도의 역사', '이집트의 역사', '전주역

In [9]:
# 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n','')
    txt = txt.replace('=','')    
    return txt 

ko_grammar_set_raw = [clean_text(x) for x in ko_grammar_set_raw]
print('Sample text : ')
print('--------------------------------------------------------------------------------------------')
print(ko_grammar_set_raw[50])
print('--------------------------------------------------------------------------------------------')

Sample text : 
--------------------------------------------------------------------------------------------
김덕현(본명은 김덕구, 1967년 4월 19일 ~ )은 대한민국의 배우이다. 충청남도 보령에서 태어나 청소년기를 보냈다. 서울예술전문대학 방송연예학과를 졸업했다. 1991년 KBS 14기 공채 탤런트로 데뷔했다. 학력 서울예술전문대학 방송연예학과 졸업 출연작  드라마 KBS2 단막드라마 《TV 손자병법》(1992년)KBS2 단막드라마 《신 손자병법》(1993년)KBS2 수목드라마 《전설의 고향》(1996년 6월 26일 ~ 1996년 9월 12일) ... 구미호/망자의 소원 역KBS2 월화 미니시리즈 《거짓말》(1998년 3월 20일 ~ 1998년 6월 2일)KBS2 드라마 《부부클리닉 사랑과 전쟁》(1999년 10월 22일 ~ 2009년 4월 17일)KBS2 월화 미니시리즈 《학교》(1999년 2월 22일 ~ 1999년 4월 13일) ... 선생님 역MBC 단막극 《MBC 베스트극장》(1999년)iTV 일일드라마 《해바라기 가족》(2001년)MBC 법정드라마 《실화극장 죄와 벌》(2003년)KBS2 주말연속극 《애정의 조건》 (2004년 3월 20일 ~ 2004년 10월 10일)SBS 일일드라마 《소풍가는 여자》 (2004년 5월 3일 ~ 2004년 10월 8일)KBS1 일일연속극 《금쪽같은 내 새끼》(2004년 6월 7일 ~ 2005년 2월 11일)KBS1 대하드라마 《불멸의 이순신》(2004년 9월 4일 ~ 2005년 8월 28일) ... 언복 역KBS1 TV소설 《고향역》(2005년 8월 29일 ~ 2006년 3월 18일) ... 최 계장 역KBS1 대하드라마 《서울 1945》(2006년 1월 7일 ~ 2006년 9월 10일) - 박헌영 비서 역KBS2 아침드라마 《아줌마가 간다》(2006년 11월 13일 ~ 2007년 5월 19일) - 김성태 역KBS2 아침드라마

문장으로 잘라 낸다

In [10]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [11]:
#Split the document into sentences
ko_grammar_sentences = []
for document in ko_grammar_set_raw:
    ko_grammar_sentences += nltk.sent_tokenize(document)

print("Num sentences:", len(ko_grammar_sentences))

Num sentences: 13551


In [12]:
ko_grammar_sentences[300]

"현재 기탄교육은 학습지로는 '기탄국어, 한글떼기, 기탄한글, 기탄수학, 기탄사고력수학, 영단어 암기 끝!, 기탄 한자 빨리따기, 기탄중국어' 등이 있다."

In [13]:
f = open("ko_sentence_set.txt", "a")
for t in ko_grammar_sentences:
    f.write(t+'\n')
f.close()

형태소 분리하여 모든 문장을 형태소 Code로 변환 한다.

In [14]:
from konlpy.tag import Twitter
twitter = Twitter()

print(twitter.pos(ko_grammar_sentences[305]))

[('백운호수', 'Noun'), ('에서', 'Josa'), ('의왕시', 'Noun'), ('와', 'Josa'), ('의왕시', 'Noun'), ('축제', 'Noun'), ('추진', 'Noun'), ('위원회', 'Noun'), ('가', 'Josa'), ('공동', 'Noun'), ('으로', 'Josa'), ('주최', 'Noun'), ('하여', 'Verb'), ('매년', 'Noun'), ('9월', 'Number'), ('에', 'Foreign'), ('이틀', 'Noun'), ('동안', 'Noun'), ('열리는', 'Verb'), ('축제', 'Noun'), ('이다', 'Josa'), ('.', 'Punctuation')]


In [15]:
# 형태소 Code table의 구성

_MAX_MORP_LENGTH = 128
_PADDING_CODE = 0  # padding code
_MISMATCH_CODE = 1 # mismatch word code               ex) @@@
_MISMATCH_WORD = '@@@' # 이거 아래에서 쓴다.

morpheme_table = {}
morp_code = _MISMATCH_CODE+1
morpheme_table['Pad'] = _PADDING_CODE 
morpheme_table['Mst'] = _MISMATCH_CODE 
for sentence in ko_grammar_sentences[:1000]:
    morphemes = twitter.pos(sentence)
    for (word,morp) in morphemes:
        if morp in morpheme_table:
            pass
        else:
            morpheme_table[morp] = morp_code
            morp_code += 1

In [16]:
print('Korean morpheme code table')
print('----------------------------------------------------------')
print('  Morpheme        Code')
print('')
for morp in morpheme_table.keys():
    print(f' {morp.ljust(15)}   {morpheme_table[morp]}')
print('----------------------------------------------------------')


Korean morpheme code table
----------------------------------------------------------
  Morpheme        Code

 Pad               0
 Mst               1
 Noun              2
 Punctuation       3
 Foreign           4
 Josa              5
 Verb              6
 Modifier          7
 Adjective         8
 Suffix            9
 Adverb            10
 Number            11
 Alpha             12
 Conjunction       13
 Determiner        14
 VerbPrefix        15
 Exclamation       16
 KoreanParticle    17
 Eomi              18
 ScreenName        19
 URL               20
----------------------------------------------------------


In [17]:
# morpheme 코드 변환기
def morpheme_encode(sentence):
    encode=[]
    morphemes = twitter.pos(sentence)
    for (word,morp) in morphemes:
        encode.append(_MISMATCH_CODE if word==_MISMATCH_WORD else morpheme_table[morp])
    return encode

code = morpheme_encode('야당이 수단으로 야당에게는 이라며 윤석열 허물어뜨렸고 수사를 군사작전을 하는 시비를 민주당 의원이 국민적 세워 화신이 나라 법위에 우려했다 될 이라며 운영할지 바람 짓밟지만 양자역학 양자역학 걱정하느냐고 누가 청와대 한다 엄포를 양자역학 양자역학 양자역학 편을 파렴치와 난관에 풀이다 대통령이 양자역학 저지하려다가')
print(f'Code length : {len(code)}')

Code length : 67


In [18]:
# 전체 형태소 코드로 변환
if DO_ALL:
    ko_grammar_set = []
    for sentence in ko_grammar_sentences:
        code = morpheme_encode(sentence)
        if len(code) <= _MAX_MORP_LENGTH:
            ko_grammar_set.append(code + [_PADDING_CODE for i in range(_MAX_MORP_LENGTH-len(code))])

    ko_grammar_set = np.asarray(ko_grammar_set)
ko_grammar_set.shape

(13352, 128)

In [19]:
from numpy import savetxt
# save to csv file
savetxt('ko_morpheme_set.csv', ko_grammar_set, delimiter=',')


# Dataset 전체 구성

In [20]:
from sentence_transformers import SentenceTransformer
from transformers import BertTokenizer

if DO_ALL:
    # embedder download...
    embedder = SentenceTransformer('xlm-r-large-en-ko-nli-ststb')

100%|██████████| 1.80G/1.80G [01:05<00:00, 27.6MB/s]


In [21]:
BATCH_SIZE = 64
BATCH_COUNT = 50

In [22]:
# 원문의 간단한 전처리
def clean_text(txt):
    txt = txt.replace('\n','')
    txt = txt.replace('=','')    
    txt = txt.replace('\"','')    
    txt = txt.replace('\'','')    
    return txt 

org_text = clean_text(org_text)

In [23]:
org_text

'옛날 어느 집에 귀여운 여자 아기가 태어났어요.아기는 무럭무럭 자라서, 예쁘고 마음씨 고운 소녀가 되었어요.그러던 어느날, 소녀의 어머니가 병이들어그만 세상을 떠나고 말았어요. 소녀의 아버지는 홀로 남은 소녀가 걱정되었어요.그래서 얼마 후 새어머니를 맞이했어요.새어머니는 소녀보다 나이가 위인 두 딸을 데리고 왔어요.그러나 새어머니와 언니들은 성질이 고약한 심술쟁이들이었어요.새어머니는 소녀가 자기 딸들보다 예쁘고 착한 게 못마땅했어요. 그런데 이번에는 아버지마저 돌아가셨어요.소녀는 하녀처럼 하루 종일 쓸고, 닦고, 집안일을 도맡아 했어요.해도 해도 끝이 없는 집안일이 힘들어 지칠때면난롯가에 앉아서 잠시 쉬곤 했지요. 엄마, 저애를 신데렐라라고 불러야겠어요.온통 재투성이잖아요. 호호호! 두 언니는 소녀를 놀려 댔어요. 어느 날, 왕궁에서 무도회가 열렸어요.신데렐라의 집에도 초대장이 왔어요. 새어머니는 언니들을 데리고 무도회장으로 떠났어요.신데렐라도 무도회에 가고 싶었어요.혼자 남은 신데렐라는 훌쩍훌쩍 울기 시작했어요.신데렐라, 너도 무도회에 가고 싶니?신데렐라가 고개를 들어보니, 마법사 할머니가 빙그레 웃고 있었어요. 내가 너를 무도회에 보내주마호박 한개와 생쥐 두마리, 도마뱀을 구해 오렴.  마법사 할머니가 주문을 외웠어요.그리고 지팡이로 호박을 건드리자, 호박이 화려한 황금 마차로 변했어요.이번에는 생쥐와 도마뱀을 건드렸어요.그랬더니 생쥐는 흰말로, 도마뱀은 멋진 마부로 변했답니다. 신데렐라의 옷도 구슬 장식이 반짝이는 예쁜 드레스로 바뀌웠어요.신데렐라, 발을 내밀어 보거라.할머니는 신데렐라에게 반짝반짝 빛나는 유리 구두를 신겨 주었어요신데렐라, 밤 열두시가 되면 모든게 처음대로 돌아간단다.황금 마차는 호박으로, 흰말은 생쥐로, 마부는 도마뱀으로 변하게 돼.그러니까 바늗시 밤 열두 시가 되기 전에 돌아와야 해. 알겠지? 왕자님도 아름다운 신데렐라에게 마음을 빼았겼어요.완자님은 무도회장에 모인 다른 아가씨들은 쳐다보지도 않고,신데렐라하고만 춤을 추었어요.신데렐

In [24]:
#dataset 다시 만듦

org_text_emb = embedder.encode([org_text])[0]
print(f'Text embedding shape : {org_text_emb.shape}')
dataset = []
for i in range(BATCH_COUNT):
    emb_batch_set = []
    cod_batch_set = []
    for j in range(BATCH_SIZE):
        emb_batch_set.append(org_text_emb)
        cod_batch_set.append(ko_grammar_set[BATCH_SIZE*i+j])

    emb_batch_set = np.asarray(emb_batch_set)
    cod_batch_set = np.asarray(cod_batch_set)
    dataset.append((emb_batch_set,cod_batch_set))

print(f'Total dataset count :{BATCH_COUNT*BATCH_SIZE}')

Text embedding shape : (1024,)
Total dataset count :3200


In [25]:
# 30번째 배치의 형태소코드셋 중 20번째꺼 확인
print(dataset[30][1][20])
print(dataset[31][1][20])

[2 9 2 5 2 2 5 6 2 2 5 6 2 2 5 2 5 6 3 2 5 2 5 2 6 2 5 6 2 5 6 6 2 8 3 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[ 2  2  5  2  5  2  5  2  2  2  5  2  8  3  2  5  2  6  6  2  5  2  5  3
 11  3 11  3 11  3 11  3 11  3 11  3 11  3  2  2  5  2  5  6  2  6  2  5
  6  3  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]


# Generator 구현

In [26]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts([org_text])
total_words = len(tokenizer.word_index)

print(f'Total token count of origin text : {total_words}')

Total token count of origin text : 276


원문을 40개의 token 으로 구성된 문장으로 요약 하는것....<br>
향후 LSTM으로 G 구성 필요

In [27]:

_MAX_GEN_TOKEN = 40
_NOISE_DIM = total_words

word_table = {}

for word,index in tokenizer.word_index.items():
    word_table[index-1] = word

current_token_len = len(word_table)



In [28]:
print('Token table of origin text')
print('---------------------------------------------')
print(' Code         Token      ')
print('')
for k in word_table.keys():
  print( f'  {str(k).ljust(8)}    {word_table[k]}')
print('---------------------------------------------')


Token table of origin text
---------------------------------------------
 Code         Token      

  0           유리
  1           구두를
  2           신데렐라는
  3           소녀가
  4           새어머니는
  5           데리고
  6           신데렐라의
  7           무도회에
  8           신데렐라가
  9           땡
  10          어느
  11          예쁘고
  12          소녀의
  13          남은
  14          그래서
  15          두
  16          왔어요
  17          언니들은
  18          이번에는
  19          해도
  20          가고
  21          신데렐라
  22          마법사
  23          할머니가
  24          도마뱀을
  25          황금
  26          발을
  27          신데렐라에게
  28          밤
  29          열두
  30          춤을
  31          구두
  32          한
  33          신데렐라를
  34          왕자님은
  35          말했어요
  36          구두의
  37          신하들은
  38          구두는
  39          옛날
  40          집에
  41          귀여운
  42          여자
  43          아기가
  44          태어났어요
  45          아기는
  46          무럭무럭
  47          자라서
  48          마음씨
  49          

In [29]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (Input,
                                     Dense, 
                                     BatchNormalization, 
                                     LeakyReLU,
                                     Softmax,
                                     Reshape, 
                                     Conv2DTranspose,
                                     Conv2D,
                                     Dropout,
                                     Flatten,
                                     Concatenate,
                                     Lambda)
import matplotlib.pyplot as plt

noise를 token_table을 통해 text로 변환하는 변환기 구현

In [30]:

b = tf.constant([1,2,3,4,5,6,11,8,9,10])
c = tf.sort(b,axis=-1,direction='DESCENDING')
wh = tf.where(tf.math.greater_equal(b,c[5-1]))[:5]
wh

<tf.Tensor: shape=(5, 1), dtype=int64, numpy=
array([[5],
       [6],
       [7],
       [8],
       [9]])>

In [31]:
import sys

def to_text(w):

    #tf.print(w)
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            text = ""
            wh = tf.where(tf.math.greater_equal(w_0r,s_w_0r[_MAX_GEN_TOKEN]))[:_MAX_GEN_TOKEN]

            for k in tf.reshape(wh,(_MAX_GEN_TOKEN,)).numpy():
                try:
                    text += word_table[k]+' '
                except Exception as ex:
                    tf.print('to_text : word_table' + str(ex),k)
                
            texts.append(text)
    except Exception as ex:
        tf.print('to_text : ' + str(ex),sys.exc_info())

    return tf.constant(texts,dtype=tf.string)

In [32]:
# to_text 함수의 test
w = tf.random.normal([3,_NOISE_DIM])
print(w.shape)
e = to_text(w)
for t in e:
    print(t.numpy().decode('utf-8'))

(3, 276)
신데렐라는 해도 마법사 구두 아기가 아기는 어머니가 홀로 새어머니와 성질이 자기 하녀처럼 닦고 집안일이 불러야겠어요 무도회장으로 싶었어요 고개를 너를 두마리 호박을 그랬더니 마부로 변했답니다 바뀌웠어요 반짝반짝 돌아간단다 마차는 마부는 변하게 돼 해 알겠지 않고 왕궁을 신어 수 ​신데렐라는 건넨 꼭 
유리 구두를 소녀가 태어났어요 아기는 병이들어그만 고약한 소녀는 집안일이 언니는 무도회가 고개를 웃고 생쥐 두마리 외웠어요 그랬더니 예쁜 보거라 빛나는 호박으로 도마뱀으로 시가 해 가는 줄도 몰랐어요 짝이 벗겨졌어요 주울 짝을 이 주인과 나라를 그때 있나요 건넨 갔어요 오래오래 행복하게 
땡 어느 소녀의 할머니가 말했어요 구두는 아기가 아기는 후 소녀보다 딸을 쉬곤 신데렐라라고 날 무도회가 싶니 보내주마호박 두마리 호박을 호박이 마차로 생쥐는 드레스로 주었어요신데렐라 되면 처음대로 돌아간단다 흰말은 시가 마음을 빼았겼어요 완자님은 다른 소리에 틈이 없었어요 가지고 보았지만한눈에 갔어요 결혼하여 


생성된 text의 embedding 변환기 구현<br>
embedding은 org_text의 embedding과 비교하여 원문과 유사하게 민들기 위한 목적

In [33]:
# 이거 이틀걸림...잘 몰라서 ㅈㄴ 헤맴

@tf.custom_gradient
def to_embedding(w):

    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 10, 1e-3)  # gradient 증폭 
        dy_arr = tf.reshape(dy,(dy.shape[0],1024,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],w.shape[1]),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],w.shape[1]))
        return dy_arr_st

    #print(w)    
    texts = []
    value = None
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            text = ""
            wh = tf.where(tf.math.greater_equal(w_0r,s_w_0r[_MAX_GEN_TOKEN]))[:_MAX_GEN_TOKEN]
            for k in tf.reshape(wh,(_MAX_GEN_TOKEN,)).numpy():
                text += word_table[k]+' '
                 
            texts.append(text)                 
        value = tf.constant(embedder.encode(texts,show_progress_bar=False),dtype=tf.float64)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad

In [34]:
# to_embedding 함수의 test
e = to_embedding(w)
for t in e:
    print(t.numpy())

[ 0.20847742 -0.24716251  0.19367166 ... -0.32198694  0.92360282
  0.32289121]
[-0.10688743 -0.04786511 -0.40968931 ... -0.28833583  1.11195648
  0.21435505]
[ 0.09529657 -0.2386384   0.08158018 ... -0.23322445  0.81238294
  0.24843544]


In [35]:
# to_compression_ratio

@tf.custom_gradient
def to_compression_ratio(w):
    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 2, 1e-3)  # gradient 증폭, 이래도 되는 건지....
        dy_arr = tf.reshape(dy,(dy.shape[0],1,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],w.shape[1]),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],w.shape[1]))        
        return dy_arr_st

    const = tf.math.reduce_sum(tf.constant([x for x in range(tf.size(w[0]))],tf.int64)) 
    #print(const)
    value = None
    compression_ratio = [] #tf.Variable()
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            wh = tf.where(tf.math.greater_equal(w_0r,s_w_0r[_MAX_GEN_TOKEN]))[:_MAX_GEN_TOKEN]
            cr = tf.math.reduce_sum(tf.reshape(wh,(_MAX_GEN_TOKEN,)))
            compression_ratio.append(cr.numpy()/const.numpy())
        #print(compression_ratio)
        value = tf.constant(compression_ratio,dtype=tf.float64)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad
    

In [36]:
# to_compression_ratio 함수의 test
e = to_compression_ratio(w)
for t in e:
    print(t.numpy())

0.14413702239789197
0.17217391304347826
0.1463768115942029


생성된 text의 morpheme code 변환기 구현<br>
morpheme code는 한국어 문장들(dataset)의 morpheme code와 비교하여 한국어 문법에 가깝게 만들기 위한 목적

In [37]:

@tf.custom_gradient
def to_morpcoding(w):

    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 2, 1e-3)  # gradient 증폭, 이래도 되는 건지....
        dy_arr = tf.reshape(dy,(dy.shape[0],_MAX_MORP_LENGTH,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],w.shape[1]),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],w.shape[1]))
        return dy_arr_st

    #print(w)    
    codes = []
    value = None
    texts = []
    try:
        s_w = tf.sort(w,axis=-1,direction='DESCENDING')
        #print(s_w)
        for w_0r,s_w_0r in zip(w,s_w):
            text = ""
            wh = tf.where(tf.math.greater_equal(w_0r,s_w_0r[_MAX_GEN_TOKEN]))[:_MAX_GEN_TOKEN]
            for k in tf.reshape(wh,(_MAX_GEN_TOKEN,)).numpy():
                text += word_table[k]+' '
                 
            texts.append(text)    
            
        for sentence in texts:
            code = morpheme_encode(sentence)
            if len(code) <= _MAX_MORP_LENGTH:
                codes.append(code + [_PADDING_CODE for i in range(_MAX_MORP_LENGTH-len(code))])
            else:
                codes.append(code[:_MAX_MORP_LENGTH])
        value = tf.constant(codes,dtype=tf.int32)
    except Exception as ex:
        tf.print('to_morpcoding :' + str(ex),sys.exc_info())

    return value, grad

In [38]:
# to_morpcoding 함수의 test
e = to_morpcoding(w)
for t in e:
    print(t.numpy())

[ 2  5  2  2  2  2  5  2  5  2  5  2  2  5  2  5  2  2  5  6  2  5  6  2
  9  5  6  2  5  2  5 14  2  2  5 13  2  5  8  6  2  9  5 10  6  2  5  2
  5  8  6  2  6  6  2  5  2  2  4  2  5  6  2  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
[ 2  2  5  2  5  6  2  5  2  5  6 10  8  2  5  2  5  2  5  2  5  2  5  6
  2 14  2  6 13  8  6  6  2  5  2  5  2  2  6  2  5  6  2  5  6  6  2  5
  2  2  5  2  5  2  8  6  6  2  8  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  0  0  0  0]
[ 2 10  2  5  2  5  2  6  2  5  2  5  2  5  2  2  5  2  5  6  2  5  2  2
  5  6  6  2 14  2  2  5  2  5  2  5  2  5  2  5  6  2  6  2  5  6  8  2
  2  5  6  6  2  5  2  2  5  2  5  8  2  5  6  2  5  6  2  6  0  0  0  0

Network 구성을 위해 사용자 정의 Layer 를 구현

In [39]:
# 이것도 잘 몰라서 하루 걸림... ㅜㅜ

from keras import backend as K
from keras.layers import Layer

#tf.executing_eagerly()

class Post_processing(Layer):

    def __init__(self, output_dim, encoder_func=None,Tout=tf.float64, **kwargs):
        self.output_dim = output_dim
        self.encoder = encoder_func
        self.Tout = Tout
        super(Post_processing, self).__init__(**kwargs)
    '''
    def build(self, input_shape):
        tf.print('build',input_shape)
        # 이 레이어에 대해 학습가능한 가중치 변수를 만듭니다.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.output_dim),
                                      initializer='uniform',
                                      trainable=True)
        super(Post_processing, self).build(input_shape)  # 끝에서 꼭 이 함수를 호출하십시오
    '''
    def call(self, input_data):
        #tf.print('Post_processing : call input_data',input_data.shape)
        value = tf.py_function(self.encoder,[input_data],Tout=self.Tout,name='encode_func')
        #print('value.shape:',value.shape)
        #value.set_shape((input_data.shape[0],self.output_dim))
        if self.output_dim > 0:
            value.set_shape((input_data.shape[0],self.output_dim))
        else:
            value.set_shape((input_data.shape[0],))
        #return tf.reshape(value,[input_data.shape[0]])  

        #value = tf.Variable((tf.zeros([input_data.shape[0],1024]) if self.Tout==tf.float64 else tf.zeros([input_data.shape[0],])),dtype=self.Tout,shape=( (input_data.shape[0],1024) if self.Tout==tf.float64 else (input_data.shape[0],)))
        #tf.py_function(self.encoder,[input_data],Tout=self.Tout)
        return value

    def compute_output_shape(self, input_shape):
        tf.print('compute_output_shape:',input_shape)
        input_shape = tensor_shape.TensorShape(input_shape).as_list()
        if self.output_dim > 0:
            return tensor_shape.TensorShape([input_shape[0], self.output_dim])
        return tensor_shape.TensorShape([input_shape[0]])

In [40]:
# 구성한 Layer의 test
e = Post_processing(1024,to_embedding,Tout=tf.float64)(w)
for c in e:
    print(c)

tf.Tensor(
[ 0.20847742 -0.24716251  0.19367166 ... -0.32198694  0.92360282
  0.32289121], shape=(1024,), dtype=float64)
tf.Tensor(
[-0.10688743 -0.04786511 -0.40968931 ... -0.28833583  1.11195648
  0.21435505], shape=(1024,), dtype=float64)
tf.Tensor(
[ 0.09529657 -0.2386384   0.08158018 ... -0.23322445  0.81238294
  0.24843544], shape=(1024,), dtype=float64)


In [41]:
# 별로 중요하지는 않지만 Lambda layer를 활용하기 위한 assert 함수 구성
def assert_layer(input_data,out_dim=None):
    #tf.print(input_data)
    #print(input_data)
    assert input_data.shape[1] == out_dim
    return input_data

In [42]:
# 드디어 generator 구현
# 효과적으로 구성된 것인지는 모르겠음... 이것은 아직 많은 연구가 필요함.
# 또한 LSTM으로 바꾸어 길이의 한게를 극복해야 할 것...

def make_generator_model(org_words):
    input = Input(shape=(org_words,), dtype='float64') 
    x1 = Dense(org_words*2, use_bias=False)(input)
    #x1 = BatchNormalization()(x1)
    x1 = LeakyReLU()(x1)
    
    x1 = Dense(org_words*4, use_bias=False)(x1)
    #x1 = BatchNormalization()(x1)
    x1 = LeakyReLU()(x1)
    
    #x1 = Dense(max_length*total_words, use_bias=False, activation='tanh')(x1)
    x1 = Dense(org_words, use_bias=False)(x1)
    x1 = Lambda(assert_layer,arguments={'out_dim':org_words})(x1)
    #x1 = Reshape((max_length, total_words))(x1)
    #x1 = BatchNormalization()(x1)
    #x1 = Softmax()(x1)        
    #x1 = MyCustomLayer(max_length*total_words)(x1)
    txt = Post_processing(0,to_text,Tout=tf.string)(x1)
    emb = Post_processing(1024,to_embedding,Tout=tf.float64)(x1)
    cmr = Post_processing(0,to_compression_ratio,Tout=tf.float64)(x1)
    cod = Post_processing(128,to_morpcoding,Tout=tf.int32)(x1)
    
    model = Model(input,[txt,emb,cmr,cod])
    
    model.summary()
    return model

generator = make_generator_model(_NOISE_DIM)

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 276)]        0                                            
__________________________________________________________________________________________________
dense (Dense)                   (None, 552)          152352      input_1[0][0]                    
__________________________________________________________________________________________________
leaky_re_lu (LeakyReLU)         (None, 552)          0           dense[0][0]                      
__________________________________________________________________________________________________
dense_1 (Dense)                 (None, 1104)         609408      leaky_re_lu[0][0]                
______________________________________________________________________________________________

In [43]:
# generator의 test
# Create a random noise and generate a sample
noise = tf.random.normal([3,_NOISE_DIM])
texts,embeddings,compratios,morpcodes = generator(noise, training=True)
print(texts.shape)
for i,txt in zip(range(len(texts)),texts):
    print(f" {i+1}> {txt.numpy().decode('utf-8')}" )
print(embeddings.shape)
print(compratios.shape)
print(morpcodes.shape)

(3,)
 1> 유리 새어머니는 어느 열두 말했어요 신하들은 옛날 아기는 고운 세상을 못마땅했어요 하루 닦고 도맡아 불러야겠어요 온통 언니는 놀려 훌쩍훌쩍 시작했어요 있었어요 오렴 지팡이로 반짝이는 할머니는 주었어요신데렐라 되면 모든게 처음대로 마부는 도마뱀으로 마음을 벗겨졌어요 틈이 짝을 주인과 보고 보았지만한눈에 그때 조용히 
 2> 어느 말했어요 신하들은 아기가 무럭무럭 고운 아버지는 홀로 못마땅했어요 그런데 닦고 도맡아 온통 소녀를 댔어요 왕궁에서 혼자 시작했어요 싶니 내가 그리고 모든게 생쥐로 마부는 그러니까 바늗시 시가 알겠지 빼았겼어요 완자님은 소리에 화들짝 빠져나가는데 짝이 틈이 임금님께 주인과 그때 한번 그 
 3> 소녀가 땡 어느 해도 신데렐라에게 구두는 옛날 자라서 고약한 딸들보다 못마땅했어요 도맡아 없는 저애를 싶었어요 시작했어요 있었어요 지팡이로 마부로 할머니는 주었어요신데렐라 처음대로 생쥐로 그러니까 시가 왕자님도 마음을 아가씨들은 왕자님과 가는 몰랐어요 시를 알리는 화들짝 짝이 주인과 나라를 오므려도 늘려도 보기에도 
(3, 1024)
(3,)
(3, 128)


# Discriminator 구현

먼저 요약을 구분하기 위한 discriminator_summ 구현

In [44]:
# 문장에 대한 embeddings를 이용하여 org_text_emb (org_text의 embedding)과의 유사도를 계산한다.
import scipy

@tf.custom_gradient
def to_similarity(w):

    def grad(dy):
        dy = tf.nn.batch_normalization(dy, 0, tf.math.reduce_std(dy), 0, 10, 1e-3)  # gradient 증폭, 이래도 되는 건지....
        dy_arr = tf.reshape(dy,(dy.shape[0],1,1))
        #tf.print(dy_arr)
        dy_arr_st = tf.image.resize(dy_arr, size=(dy.shape[0],1024),method=tf.image.ResizeMethod.BILINEAR)
        dy_arr_st = tf.reshape(dy_arr_st,shape=(dy.shape[0],1024))
        return dy_arr_st

    similarities = []
    value = None
    try:
        for embedding in w:
            distances = scipy.spatial.distance.cdist([embedding], [org_text_emb], "cosine")[0]
            similarities.append(distances[0])
            
        value = tf.constant(similarities,dtype=tf.float64)
    except Exception as ex:
        tf.print(ex,sys.exc_info())

    return value, grad


In [45]:
def make_discriminator_model():
    input_emb = Input(shape=(1024,), dtype='float64') 
    x1 = Post_processing(0,to_similarity,Tout=tf.float64)(input_emb)
    x1 = Reshape((1,))(x1)    
    #x1 = Dense(1024*2)(input_emb)
    #x1 = BatchNormalization()(x1)
    #x1 = LeakyReLU()(x1)

    
    input_cmp = Input(shape=(), dtype='float64') 
    imput_mrp = Input(shape=(_MAX_MORP_LENGTH,), dtype='int32')

    x2 = Reshape((1,))(input_cmp)
    x3 = Dense(256)(imput_mrp)
    #x3 = BatchNormalization()(x3)
    x3 = LeakyReLU()(x3)

    x3 = Dense(256*3)(x3)
    #x3 = BatchNormalization()(x3)
    x3 = LeakyReLU()(x3)

    concatted = Concatenate(axis=1)([x1, x2, x3])
    x4 = Flatten()(concatted)
    x4 = Dense(64, use_bias=False)(x4)
    #x4 = BatchNormalization()(x4)
    x4 = LeakyReLU()(x4)
    x4 = Dense(1)(x4)
    
    model = Model([input_emb,input_cmp,imput_mrp],x4)
    
    model.summary()
    
    return model

discriminator = make_discriminator_model()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_4 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
dense_3 (Dense)                 (None, 256)          33024       input_4[0][0]                    
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 1024)]       0                                            
__________________________________________________________________________________________________
leaky_re_lu_2 (LeakyReLU)       (None, 256)          0           dense_3[0][0]                    
____________________________________________________________________________________________

In [46]:
# discriminator test

predict = discriminator([embeddings,compratios,morpcodes])
print(predict)

tf.Tensor(
[[-0.75045085]
 [-1.6413989 ]
 [-3.4098363 ]], shape=(3, 1), dtype=float32)


# GAN 을 이용한 loss 의 gradient 구현 --> 빡심

In [47]:
# This method returns a helper function to compute cross entropy loss
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

generator_optimizer = tf.keras.optimizers.Adam(1e-4)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-4)

In [48]:
# 디짐
@tf.function
def train_step(real_embedding,real_morpcoding):
  
    # 1 - Create a random noise to feed it into the model
    # for the text generation
    noise = tf.random.normal([BATCH_SIZE, _NOISE_DIM])
    
    # 2 - Generate text and calculate loss values
    # GradientTape method records operations for automatic differentiation.
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        texts,embeddings,compratios,morpcodes = generator(noise, training=True)
        #embeddings = generator(noise, training=True)
        # real에 가까우려면 discriminator의 학습은 real_embedding 이 zero (0)에 가깝게 학습시켜야 함.
        # 하지만 압축율의 개념으로는 본래는 ones (1)가 맞음.
        real_output = discriminator([real_embedding,np.zeros(len(real_embedding)),real_morpcoding], training=True)
        fake_output = discriminator([embeddings,compratios,morpcodes], training=True)

        #tf.print('train_step : embeddings.shape=',embeddings.shape)
        gen_loss = generator_loss(fake_output)
        #tf.print('train_step : gen_loss=',gen_loss)
        disc_loss = discriminator_loss(real_output, fake_output)
        #tf.print('train_step : disc_loss=',disc_loss)

    # 3 - Calculate gradients using loss values and model variables
    # "gradient" method computes the gradient using 
    # operations recorded in context of this tape (gen_tape and disc_tape).
    
    # It accepts a target (e.g., gen_loss) variable and 
    # a source variable (e.g.,generator.trainable_variables)
    # target --> a list or nested structure of Tensors or Variables to be differentiated.
    # source --> a list or nested structure of Tensors or Variables.
    # target will be differentiated against elements in sources.

    # "gradient" method returns a list or nested structure of Tensors  
    # (or IndexedSlices, or None), one for each element in sources. 
    # Returned structure is the same as the structure of sources.
    
    gradients_of_discriminator = disc_tape.gradient(disc_loss, 
                                                discriminator.trainable_variables)
    #tf.print('train_step : gradients_of_discriminator=',gradients_of_discriminator)   
    noise = tf.random.normal([BATCH_SIZE, _NOISE_DIM])
    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        texts,embeddings,compratios,morpcodes = generator(noise, training=True)
        #embeddings = generator(noise, training=True)
        #real_output = discriminator(real_embedding, training=True)
        fake_output = discriminator([embeddings,compratios,morpcodes], training=True)

        #tf.print('train_step : embeddings.shape=',embeddings.shape)
        gen_loss = generator_loss(fake_output)
        #tf.print('train_step : gen_loss=',gen_loss)
        #disc_loss = discriminator_loss(real_output, fake_output)
        #tf.print('train_step : disc_loss=',disc_loss)

    gradients_of_generator = gen_tape.gradient(gen_loss, 
                                               generator.trainable_variables)
    #tf.print('train_step : gradients_of_generator=',gradients_of_generator)
 

    # 4 - Process  Gradients and Run the Optimizer
    # "apply_gradients" method processes aggregated gradients. 
    # ex: optimizer.apply_gradients(zip(grads, vars))
    """
    Example use of apply_gradients:
    grads = tape.gradient(loss, vars)
    grads = tf.distribute.get_replica_context().all_reduce('sum', grads)
    # Processing aggregated gradients.
    optimizer.apply_gradients(zip(grads, vars), experimental_aggregate_gradients=False)
    """
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))
    #tf.print('train_step : after discriminator_optimizer')    

In [49]:
EPOCHS = 5
# 요약문 생성의 확인을 위해 10개의 문장을 생성하고 train과정에서 각 epoch마다 변화를 확인한다.
seed = tf.random.normal([10, _NOISE_DIM])

In [50]:
# 생성된 문장의 원문 유사도를 측정하기 위한 함수

import scipy
#print(doc_emb)
def similarity_score(queries,org_embedding):

    total_score = 0
    query_embeddings = embedder.encode(queries,show_progress_bar=False)
    for query, query_embedding in zip(queries, query_embeddings):
        distances = scipy.spatial.distance.cdist([query_embedding], [org_embedding], "cosine")[0]
        results = zip(range(len(distances)), distances)
        for idx, distance in results:
            total_score += 1-distance
    return total_score

queries = []

texts,embeddings,compratios,morpcodes = generator(seed,training=False)

#count = 0
for t in texts:
    summary_text = t.numpy().decode('utf-8')
    queries.append(summary_text)
print('Similarity score:',str(similarity_score(queries,org_text_emb)))


Similarity score: 5.63828260513954


In [51]:
# Print iterations progress
class ProgressBar:
    # pb = ProgressBar(total=20, prefix = 'Epoch 1')
    def __init__(self,total=20, prefix = '', suffix = '', decimals = 1, length = 20, fill = '█', printEnd = "\r"):
        self.total = total
        self.prefix = prefix
        self.suffix = suffix
        self.decimals = decimals
        self.length = length
        self.fill = fill
        self.printEnd = printEnd
        self.ite = 0
    # pb.printProgress(1,'~~~~')
    def printProgress(self,iteration, text):
        self.ite += iteration
        percent = ("{0:." + str(self.decimals) + "f}").format(100 * (self.ite / float(self.total)))

        filledLength = int(self.length * self.ite // self.total)
        bar = self.fill * filledLength + '-' * (self.length - filledLength)
        print(f'\r{self.prefix} |{bar}| {percent}% {self.suffix}  {text}', end="", flush=True)
        # Print New Line on Complete
        if self.ite == self.total: 
            print()

In [52]:
import time
from IPython import display # A command shell for interactive computing in Python.
import re

def train(dataset, epochs):
    print('Start with seed text.')
    seed = tf.random.normal([10, _NOISE_DIM])
    print('-------------------------------------------------------')
    texts,embeddings,compratios,morpcodes = generator(seed,training=False)
    for i,t in  zip(range(len(texts)),texts):
        summary_text = t.numpy().decode('utf-8')
        print(f'{i+1} > {summary_text}')        
    print('-------------------------------------------------------')
    print('')

    # A. For each epoch, do the following:
    for epoch in range(epochs):
        start = time.time()
        pb = ProgressBar(total=BATCH_COUNT, prefix = f'Epoch:{str(epoch+1)}/{epochs}')
        pb.printProgress(0,'Start batch.')
        # 1 - For each batch of the epoch, 
        for batch_num,(emb_batch_set,cod_batch_set) in zip(range(len(dataset)),dataset):
            # 1.a - run the custom "train_step" function
            # we just declared above
            #print(image_batch.shape)
            train_step(emb_batch_set,cod_batch_set)
            texts,embeddings,compratios,morpcodes = generator(seed,training=False)
            pb.printProgress(+1,f"Time for batch {batch_num + 1}/{BATCH_COUNT} is {int(time.time()-start)} sec, generated text:{texts[0].numpy().decode('utf-8')}")
        # 4 - Print out the completed epoch no. and the time spent
        #print (f'Time for epoch {} is {} sec'.format(epoch + 1, time.time()-start))
        #texts,embeddings,compratios,morpcodes = generator(seed,training=False)
        count = 0
        queries = []
        texts,embeddings,compratios,morpcodes = generator(seed,training=False)
        for i,t in  zip(range(len(texts)),texts):
            summary_text = t.numpy().decode('utf-8')
            print(f'{i+1} > {summary_text}')
            queries.append(summary_text)
            c = [m.start() for m in re.finditer(_MISMATCH_WORD, summary_text)]
            count += len(c)
        print(f'Mismatch count:{count} Similarity score:{str(similarity_score(queries,org_text_emb))}')
        print('')

In [53]:
if DO_ALL:
    EPOCHS = 30
    train(dataset, EPOCHS)

Start with seed text.
-------------------------------------------------------
1 > 구두를 신데렐라의 어느 남은 가고 황금 딸을 자기 딸들보다 종일 도맡아 저애를 온통 언니들을 신데렐라도 시작했어요 들어보니 주문을 호박을 반짝이는 할머니는 되면 처음대로 마부는 도마뱀으로 왕자님도 아름다운 마음을 추었어요 몰랐어요 벽시계가 시를 알리는 뛰쫓아오던 주인과 돌아다녔어요 보기에도 저도 한번 수 
2 > 신데렐라의 소녀의 두 언니들은 황금 신데렐라를 구두는 귀여운 아기는 아버지는 고약한 딸들보다 못마땅했어요 잠시 쉬곤 놀려 댔어요 무도회장으로 시작했어요 빙그레 주문을 건드렸어요 장식이 할머니는 신겨 주었어요신데렐라 처음대로 마부는 도마뱀으로 해 아름다운 마음을 아가씨들은 시를 알리는 왕궁을 틈이 뛰쫓아오던 수 완자님과 
3 > 신데렐라의 어느 신데렐라 도마뱀을 황금 신데렐라에게 열두 말했어요 구두는 옛날 고운 말았어요 딸들보다 못마땅했어요 지칠때면난롯가에 불러야겠어요 온통 놀려 언니들을 시작했어요 싶니 있었어요 그리고 지팡이로 생쥐는 할머니는 주었어요신데렐라 되면 생쥐로 도마뱀으로 알겠지 마음을 추었어요 허둥지둥 짝이 임금님께 한번 신하게 건넨 완자님과 
4 > 새어머니는 데리고 무도회에 신데렐라가 언니들은 신데렐라 열두 한 여자 무럭무럭 고운 후 못마땅했어요 그런데 도맡아 불러야겠어요 놀려 신데렐라도 너도 지팡이로 마부로 장식이 할머니는 신겨 주었어요신데렐라 열두시가 모든게 마부는 도마뱀으로 바늗시 빼았겼어요 시간 시를 알리는 화들짝 벗겨졌어요 틈이 주인과 결혼하겠어요 찾아 
5 > 신데렐라의 이번에는 신데렐라에게 신데렐라를 아기가 세상을 얼마 맞이했어요 성질이 딸들보다 못마땅했어요 그런데 종일 없는 잠시 온통 집에도 시작했어요 그리고 호박이 반짝이는 예쁜 할머니는 처음대로 돌아간단다 호박으로 흰말은 시가 왕자님도 빼았겼어요 몰랐어요 벽시계가 알리는 소리에 화들짝 틈이 임금님께 보았지만한눈에 보기에도 저도

KeyboardInterrupt: ignored